In [14]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [15]:
import requests
import time

import pandas as pd
from bs4 import BeautifulSoup as bs
import html5lib

In [16]:
def make_request_with_retry(url, max_retries=5, backoff_factor=2):
    retries = 0
    while retries < max_retries:
        response = requests.get(url)
        if response.status_code < 300:  # Successful response
            return response
        elif response.status_code == 429:  # Too many requests
            print(f"Rate limited. Retrying in {backoff_factor ** retries} seconds.")
            time.sleep(backoff_factor ** retries)
            retries += 1
        else:
            print(f"Request failed with status code {response.status_code}")
            return None

In [17]:
url = 'https://www.vivareal.com.br/aluguel/santa-catarina/florianopolis/?pagina={}'

In [18]:
df = pd.DataFrame(
    columns=[
        'descricao',
        'endereco',
        'valor',
        'condominio',
        'area',
        'qtd_banheiros',
        'qtd_quartos',
        'qtd_vagas',
        'link'
             ]
)

In [19]:
ret = make_request_with_retry(url.format(1))
soup = bs(ret.text, features="html5lib")
houses_num = int(soup.find('strong',{'class':'results-summary__count'}).text.strip().replace('.',''))

In [20]:
ret

<Response [200]>

In [21]:
df

,descricao,endereco,valor,condominio,area,qtd_banheiros,qtd_quartos,qtd_vagas,link


In [22]:
i = 1

In [24]:
while df.shape[0] < houses_num:
    # Mesma requisição de antes, porém retornando todas as tags article
    ret = make_request_with_retry(url.format(i))
    soup = bs(ret.text, features="html5lib")
    houses = soup.find_all('article', {'class': 'property-card__container js-property-card'})
    
    # Loop, itera todas as tags article tentando encontrar a tag correspondente ao valor desejado, se não encontra, define None 
    for house in houses:
        try:
            descricao = house.find('span', {'class': 'js-card-title'}).text.strip()
        except:
            descricao = None
        try:
            endereco = house.find('span', {'class': 'property_card__address'}).text.strip()
        except:
            endereco = None
        try:
            valor = house.find('div', {'class': 'property-card__price'}).text.strip()
        except:
            valor = None
        try:
            condominio = house.find('strong', {'class': 'js-condo-price'}).text.strip()
        except:
            condominio = None
        try:
            area = house.find('span', {'class': 'js-property-card-detail-area'}).text.strip().extract('(\d+)', expand=False)
        except:
            area = None
        try:
            qtd_banheiros = house.find('li', {'class': 'property-card__detail-bathroom'}).text.strip()
            qtd_banheiros = None
        try:
            qtd_quartos = house.find('li', {'class': 'property-card__detail-room'}).text.strip()
        except:
            qtd_quartos = None
        try:
            qtd_vagas = house.find('li', {'class': 'property-card__detail-garage'}).text.strip()
        except:
            qtd_vagas = None
        try:
            wlink = 'https://vivareal.com.br' + house.find('a', {'class': 'property-card__labels-container'})['href']
        except:
            wlink = None
        
        # Define uma lista com o valor das variáveis acima e guarda na última posição do dataframe.
        df.loc[df.shape[0]] = [
            descricao,
            endereco,
            valor,
            condominio,
            area,
            qtd_banheiros,
            qtd_quartos,
            qtd_vagas,
            wlink
        ]
    # Atualiza a variável de controle e printa o status da execução do loop
    i += 1
    print(f'i = {i} \t \tsize = {df.shape[0]}')

i = 2 	 	size = 36
i = 3 	 	size = 72
i = 4 	 	size = 108
i = 5 	 	size = 144
i = 6 	 	size = 180
i = 7 	 	size = 216
i = 8 	 	size = 252
i = 9 	 	size = 288
i = 10 	 	size = 324
i = 11 	 	size = 360
i = 12 	 	size = 396
i = 13 	 	size = 432
i = 14 	 	size = 468
i = 15 	 	size = 504
i = 16 	 	size = 540
i = 17 	 	size = 576
i = 18 	 	size = 612
i = 19 	 	size = 648
i = 20 	 	size = 684
i = 21 	 	size = 720
i = 22 	 	size = 756
i = 23 	 	size = 792
i = 24 	 	size = 828
i = 25 	 	size = 864
i = 26 	 	size = 900
i = 27 	 	size = 936
i = 28 	 	size = 972
i = 29 	 	size = 1008
i = 30 	 	size = 1044
i = 31 	 	size = 1080
i = 32 	 	size = 1116
i = 33 	 	size = 1152
i = 34 	 	size = 1188
i = 35 	 	size = 1224
i = 36 	 	size = 1260
i = 37 	 	size = 1296
i = 38 	 	size = 1332
i = 39 	 	size = 1368
i = 40 	 	size = 1404
i = 41 	 	size = 1440
i = 42 	 	size = 1476
i = 43 	 	size = 1512
i = 44 	 	size = 1548
i = 45 	 	size = 1584
i = 46 	 	size = 1620
i = 47 	 	size = 1656
i = 48 	 	size = 1692
i =

In [29]:
df.sample(50)

,descricao,endereco,valor,condominio,area,qtd_banheiros,qtd_quartos,qtd_vagas,link
1042,"Apartamento com 2 Quartos para Aluguel, 40m²",None,R$ 1.950 /Mês,R$ 140,None,None,None,None,https://vivareal.com.br/imovel/apartamento-2-q...
3614,"Apartamento com 2 Quartos para Aluguel, 82m²",None,R$ 4.200 /Mês,R$ 400,None,None,None,None,https://vivareal.com.br/imovel/apartamento-2-q...
3679,"Apartamento com 2 Quartos para Aluguel, 57m²",None,R$ 2.800 /Mês,R$ 550,None,None,None,None,https://vivareal.com.br/imovel/apartamento-2-q...
6773,"Apartamento com 2 Quartos para Aluguel, 80m²",None,R$ 2.000 /Mês,R$ 150,None,None,None,None,https://vivareal.com.br/imovel/apartamento-2-q...
5278,"Apartamento com 2 Quartos para Aluguel, 72m²",None,R$ 2.600 /Mês Preço abaixo do mercado,R$ 709,None,None,None,None,https://vivareal.com.br/imovel/apartamento-2-q...
5000,"Sala/Conjunto para Aluguel, 37m²",None,R$ 1.500 /Mês,None,None,None,None,None,https://vivareal.com.br/imovel/sala-comercial-...
3737,"Casa de condomínio com 3 Quartos para Aluguel,...",None,R$ 7.200 /Mês Preço abaixo do mercado,R$ 530,None,None,None,None,https://vivareal.com.br/imovel/casa-de-condomi...
6181,"Galpão/Depósito/Armazém para Aluguel, 240m²",None,R$ 7.000 /Mês,None,None,None,None,None,https://vivareal.com.br/imovel/galpao-deposito...
2132,"Apartamento com 2 Quartos para Aluguel, 87m²",None,R$ 5.600 /Mês,R$ 919,None,None,None,None,https://vivareal.com.br/imovel/apartamento-2-q...
4454,"Casa com 2 Quartos para Aluguel, 135m²",None,R$ 4.900 /Mês,None,None,None,None,None,https://vivareal.com.br/imovel/casa-2-quartos-...
